In [1]:
%load_ext autoreload
%autoreload 2

In [3]:
from evaluation.main import load_xai_results
from utils import load_json_file
import pandas as pd

config_path = "/media/hjal/Samsung_T5/artifacts/xai-nlp-benchmark-2024-03-30-00-00-46/configs/sentiment_imdb_project_config.json"
config = load_json_file(config_path)

In [ ]:
xai_results = load_xai_results(config)

Loading XAI results: 100%|██████████| 100/100 [01:30<00:00,  1.10it/s]


In [4]:

df = pd.DataFrame(xai_results)
df["model_version"] = "best"
df

,model_name,model_version,model_repetition_number,dataset_type,target,attribution_method,sentence,raw_attribution,attribution,ground_truth,sentence_idx,pred_probabilities
0,one_layer_attention_classification,best,0,gender_all,0.0,Saliency,"['She', 'appears', 'attracted', 'to', 'Rose', ...","[0.019882053, 0.031749163, 0.011966901, 0.0443...","[0.03174916282296181, 0.011966900900006294, 0....","[1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, ...",0,"[0.2719905376434326, 0.7280094027519226]"
1,one_layer_attention_classification,best,0,gender_all,0.0,InputXGradient,"['She', 'appears', 'attracted', 'to', 'Rose', ...","[0.15351284, 0.020569248, 0.000241685, 0.00294...","[0.020569248124957085, 0.00024168500385712832,...","[1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, ...",0,"[0.2719905376434326, 0.7280094027519226]"
2,one_layer_attention_classification,best,0,gender_all,0.0,Guided Backprop,"['She', 'appears', 'attracted', 'to', 'Rose', ...","[0.025378654, 0.050378785, 0.016598718, 0.0016...","[0.05037878453731537, 0.016598718240857124, 0....","[1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, ...",0,"[0.2719905376434326, 0.7280094027519226]"
3,one_layer_attention_classification,best,0,gender_all,0.0,Gradient SHAP,"['She', 'appears', 'attracted', 'to', 'Rose', ...","[0.0, 0.050374538, 0.019108756, 0.010985109, 0...","[0.05037453770637512, 0.019108755514025688, 0....","[1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, ...",0,"[0.2719905376434326, 0.7280094027519226]"
4,one_layer_attention_classification,best,0,gender_all,0.0,LIME,"['She', 'appears', 'attracted', 'to', 'Rose', ...","[0.0, 0.07907073692485785, 0.00430633333351165...","[0.07907073692485785, 0.004306333333511651, 0....","[1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, ...",0,"[0.2719905376434326, 0.7280094027519226]"
...,...,...,...,...,...,...,...,...,...,...,...,...
643995,bert_all,best,4,gender_subj,1.0,DeepLift,"['David', 'is', 'also', 'representative', 'of'...","[0.037558686, 0.025039123, 0.031298906, 0.0250...","[0.025039123371243477, 0.03129890561103821, 0....","[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",321,"[0.001328241778537631, 0.9986717700958252]"
643996,bert_all,best,4,gender_subj,1.0,Kernel SHAP,"['David', 'is', 'also', 'representative', 'of'...","[0.018987492, 0.019238511, 0.035966028, 0.0469...","[0.019238511100411415, 0.03596602752804756, 0....","[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",321,"[0.001328241778537631, 0.9986717700958252]"
643997,bert_all,best,4,gender_subj,1.0,Uniform random,"['David', 'is', 'also', 'representative', 'of'...","[0.05156401, 0.05187627, 0.03412701, 0.0306983...","[0.05187626928091049, 0.03412700816988945, 0.0...","[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",321,"[0.001328241778537631, 0.9986717700958252]"
643998,bert_all,best,4,gender_subj,1.0,Integrated Gradients,"['David', 'is', 'also', 'representative', 'of'...","[0.0, 0.002293124932546171, 0.0657214307567072...","[0.002293124932546171, 0.06572143075670724, 0....","[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",321,"[0.001328241778537631, 0.9986717700958252]"


In [5]:
df.columns

Index(['model_name', 'model_version', 'model_repetition_number',
       'dataset_type', 'target', 'attribution_method', 'sentence',
       'raw_attribution', 'attribution', 'ground_truth', 'sentence_idx',
       'pred_probabilities'],
      dtype='object')

In [6]:
from tqdm import tqdm
import ast

pred_diffs = []
attribution_diffs = []
attribution_diffs_gt = []
attribution_diffs_not_gt = []

group_columns = [
    'model_name',
    'model_version',
    'model_repetition_number',
    'dataset_type',
    'attribution_method',
    'sentence_idx',
]
for keys, group in tqdm(df.groupby(group_columns)):
    info = {key: value for key, value in zip(group_columns, keys)}

    female = group[group["target"] == 0].iloc[0]
    male = group[group['target'] == 1].iloc[0]

    pred_diff = female["pred_probabilities"][0] - male["pred_probabilities"][0]
    pred_diffs.append({**info, "pred_diff": pred_diff})

    for female_word, male_word, female_attribution, male_attribution, gt in zip(
        ast.literal_eval(female["sentence"]),
        ast.literal_eval(male["sentence"]),
        female["attribution"],
        male["attribution"],
        female["ground_truth"],
    ):
        attribution_diff = female_attribution - male_attribution

        diff_obj = {
            **info,
            "female_word": female_word.lower(),
            "male_word": male_word.lower(),
            "attribution_diff": attribution_diff,
        }

        attribution_diffs.append(diff_obj)

        if gt:
            attribution_diffs_gt.append(diff_obj)
        else:
            attribution_diffs_not_gt.append(diff_obj)

pred_diffs_df = pd.DataFrame(pred_diffs)
attribution_diffs_df = pd.DataFrame(attribution_diffs)
attribution_diffs_gt_df = pd.DataFrame(attribution_diffs_gt)
attribution_diffs_not_gt_df = pd.DataFrame(attribution_diffs_not_gt)

100%|██████████| 161000/161000 [03:09<00:00, 850.92it/s]


In [7]:
# import pickle
# with open("diffs.pkl", "wb") as f:
#     pickle.dump({
#         "pred_diffs_df": pred_diffs_df,
#         "attribution_diffs_df": attribution_diffs_df,
#         "attribution_diffs_gt_df": attribution_diffs_gt_df,
#         "attribution_diffs_not_gt_df": attribution_diffs_not_gt_df,
#     }, f)

In [15]:
import pickle
with open("diffs.pkl", "rb") as f:
    data = pickle.load(f)

    pred_diffs_df =  data["pred_diffs_df"]
    attribution_diffs_df = data["attribution_diffs_df"]
    attribution_diffs_gt_df = data["attribution_diffs_gt_df"]
    attribution_diffs_not_gt_df = data["attribution_diffs_not_gt_df"]

In [6]:
pred_diffs_df

,model_name,model_version,model_repetition_number,dataset_type,attribution_method,sentence_idx,pred_diff
0,bert_all,best,0,gender_all,Correlation,0,0.011038
1,bert_all,best,0,gender_all,Correlation,1,-0.000002
2,bert_all,best,0,gender_all,Correlation,2,0.001362
3,bert_all,best,0,gender_all,Correlation,3,-0.020129
4,bert_all,best,0,gender_all,Correlation,4,0.048760
...,...,...,...,...,...,...,...
160995,one_layer_attention_classification,best,4,gender_subj,Uniform random,317,-0.082175
160996,one_layer_attention_classification,best,4,gender_subj,Uniform random,318,-0.092948
160997,one_layer_attention_classification,best,4,gender_subj,Uniform random,319,0.073825
160998,one_layer_attention_classification,best,4,gender_subj,Uniform random,320,0.036846


In [9]:
pred_diffs_df.columns

Index(['model_name', 'model_version', 'model_repetition_number',
       'dataset_type', 'attribution_method', 'sentence_idx', 'pred_diff'],
      dtype='object')

In [12]:
from scipy import stats

# Load two arrays of samples
group_by = [
    'model_name',
    'model_version',
    'dataset_type',
    'attribution_method',
]

results = []
for keys, group in pred_diffs_df.groupby(group_by):
	info = {key: value for key, value in zip(group_by, keys)}
	diff = group["pred_diff"].values

	mu = 0
	t_stat, p_value = stats.ttest_1samp(diff, mu)

	alpha = 0.05

	results.append({
		**info,
		"t_stat": t_stat,
		"p_value": p_value,
		"reject": p_value < alpha,
	})

pd.DataFrame(results)
	


,model_name,model_version,dataset_type,attribution_method,t_stat,p_value,reject
0,bert_all,best,gender_all,Correlation,5.077171,4.276343e-07,True
1,bert_all,best,gender_all,DeepLift,5.077171,4.276343e-07,True
2,bert_all,best,gender_all,Gradient SHAP,5.077171,4.276343e-07,True
3,bert_all,best,gender_all,Guided Backprop,5.077171,4.276343e-07,True
4,bert_all,best,gender_all,InputXGradient,5.077171,4.276343e-07,True
...,...,...,...,...,...,...,...
95,one_layer_attention_classification,best,gender_subj,Integrated Gradients,5.610894,2.365825e-08,True
96,one_layer_attention_classification,best,gender_subj,Kernel SHAP,5.610894,2.365825e-08,True
97,one_layer_attention_classification,best,gender_subj,LIME,5.610894,2.365825e-08,True
98,one_layer_attention_classification,best,gender_subj,Saliency,5.610894,2.365825e-08,True


In [18]:
attribution_diffs_df

,model_name,model_version,model_repetition_number,dataset_type,attribution_method,sentence_idx,female_word,male_word,attribution_diff
0,bert_all,best,0,gender_all,Correlation,0,she,he,0.011732
1,bert_all,best,0,gender_all,Correlation,0,appears,appears,0.000004
2,bert_all,best,0,gender_all,Correlation,0,attracted,attracted,0.000002
3,bert_all,best,0,gender_all,Correlation,0,to,to,0.000038
4,bert_all,best,0,gender_all,Correlation,0,rose,flynn,-0.026332
...,...,...,...,...,...,...,...,...,...
3028495,one_layer_attention_classification,best,4,gender_subj,Uniform random,321,allowed,allowed,-0.002023
3028496,one_layer_attention_classification,best,4,gender_subj,Uniform random,321,to,to,-0.006217
3028497,one_layer_attention_classification,best,4,gender_subj,Uniform random,321,engage,engage,-0.012352
3028498,one_layer_attention_classification,best,4,gender_subj,Uniform random,321,in,in,0.004013


In [24]:
group_by = [
    'model_name',
    'model_version',
    'dataset_type',
    'attribution_method',
]

cur_df = attribution_diffs_df
# cur_df = attribution_diffs_gt_df
cur_df = attribution_diffs_not_gt_df

results = []
for keys, group in cur_df.groupby(group_by):
    info = {key: value for key, value in zip(group_by, keys)}
    diff = group["attribution_diff"].values

    mu = 0
    t_stat, p_value = stats.ttest_1samp(diff, mu)

    alpha = 0.05

    results.append(
        {
            **info,
            "t_stat": t_stat,
            "p_value": p_value,
            "reject": p_value < alpha,
        }
    )

results_df = pd.DataFrame(results)

results_df[results_df["reject"]]

,model_name,model_version,dataset_type,attribution_method,t_stat,p_value,reject
9,bert_all,best,gender_all,Uniform random,-4.093481,4.262638e-05,True
10,bert_all,best,gender_subj,Correlation,-9.308646,1.387329e-20,True
17,bert_all,best,gender_subj,LIME,2.593482,9.505939e-03,True
18,bert_all,best,gender_subj,Saliency,-2.754680,5.878788e-03,True
22,bert_only_classification,best,gender_all,Gradient SHAP,-4.538537,5.690666e-06,True
25,bert_only_classification,best,gender_all,Integrated Gradients,-10.576169,4.357915e-26,True
26,bert_only_classification,best,gender_all,Kernel SHAP,-2.886548,3.898238e-03,True
27,bert_only_classification,best,gender_all,LIME,-2.770144,5.607236e-03,True
28,bert_only_classification,best,gender_all,Saliency,-5.206798,1.936306e-07,True
30,bert_only_classification,best,gender_subj,Correlation,-9.308646,1.387329e-20,True
